In [2]:
import os 
import pandas as pd
import pickle
import re
from operator import itemgetter
import random
import seaborn as sns
import matplotlib.ticker as ticker
from matplotlib import pyplot as plt
%matplotlib inline

# Define os parametros necessarios para o teste

class Parametros:
    def __init__(self, seed, length, a, b, n_calls, band, file, modulacao):
        self.seed = seed
        self.length = length
        self.a = a
        self.b = b
        self.n_calls = n_calls
        self.band = band
        self.file = file
        self.modulacao = modulacao
        
# Lê arquivo de data utilizado
        
    def gera_data(self):
        a_file = open(self.file, "rb")
        return pickle.load(a_file)
        
# Gera o vetor de SNR
        
    def gera_vetor_snr(self):
        random.seed(self.seed)
        sinalRuido = []
        for i in range(0, self.length):
            sinalRuido.append(round(random.uniform(self.a, self.b), 1))
        return sinalRuido
    
# Calcula a banda requerida para cada chamada
    
    def calcula_banda_requerida(self, rate):
        codecRate = 64 # Kbps
        codecPayload = 160  # Bytes
        pps = 50 # milisegundos
        cabecalhoIP = 40 # Bytes 
        return ((((codecPayload * rate + cabecalhoIP) * 8) * pps) / 1000) * self.n_calls
    
# Busca o resultado no dataset baseado nos parametros de teste 
    
    def busca_resultados(self, snr):
        availableConfigs = []
        bandwidth_results = []

        for mod, param in self.gera_data().items():
            if mod == self.modulacao:
                if snr in param.keys():
                    for tupla in param.get(snr):
                        rate = tupla[3]
                        if self.calcula_banda_requerida(rate) <= self.band:
                            availableConfigs.append(tupla)
        return availableConfigs
        
# Gera os gráficos de resultados de teste
        
    def gera_graficos():
        pass
    
    
class Teste(Parametros):
#     def __init__(self, modulacao, band):
#         super().__init__(modulacao, band)


    def modelo_adaptativo(self):
#         Necessita corrigir!!!
        results = []
        for snr in self.gera_vetor_snr():
            value = sorted(sorted(self.busca_resultados(snr), key=lambda element: element[3]), 
                key=lambda element: element[0], reverse=True)[0]
            results.append(value)
        return results
            

    def modelo_fixo(self, code):
        results = []
        for snr in self.gera_vetor_snr():
            for element in self.busca_resultados(snr):
                if element[2] == code:
                    value = element
                    results.append(value)
        return results
        
        

In [3]:
# Args: | Seed | Taanho Vetor SNR | Valor inicial | Valor final | Arquivo de dados | Modulação |
resultado = Teste(0, 10, 3, 12, 1, 100, "data.pkl", 'BPSK')

In [4]:
resultado.modelo_adaptativo()

[(4.5, 0.0, 'RS(255, 247)', 1.0323886639676114),
 (4.5, 0.0, 'RS(255, 247)', 1.0323886639676114),
 (4.5, 0.0, 'RS(255, 225)', 1.1333333333333333),
 (4.5, 0.0, 'RS(255, 205)', 1.2439024390243902),
 (4.5, 0.0, 'RS(255, 225)', 1.1333333333333333),
 (4.5, 0.0, 'RS(255, 225)', 1.1333333333333333),
 (4.5, 0.0, 'RS(255, 247)', 1.0323886639676114),
 (4.5, 0.0, 'RS(240, 200)', 1.2),
 (4.5, 0.0, 'RS(255, 225)', 1.1333333333333333),
 (4.5, 0.0, 'RS(255, 247)', 1.0323886639676114)]

In [5]:
resultado.modelo_fixo('RS(255, 247)')

[(4.5, 0.0, 'RS(255, 247)', 1.0323886639676114),
 (4.5, 0.0, 'RS(255, 247)', 1.0323886639676114),
 (2.1989, 0.00015428, 'RS(255, 247)', 1.0323886639676114),
 (1.0705, 0.0048315, 'RS(255, 247)', 1.0323886639676114),
 (4.2875, 1.6333e-06, 'RS(255, 247)', 1.0323886639676114),
 (1.6405, 0.00034509, 'RS(255, 247)', 1.0323886639676114),
 (4.5, 0.0, 'RS(255, 247)', 1.0323886639676114),
 (1.034, 0.0030761, 'RS(255, 247)', 1.0323886639676114),
 (3.6205, 1.5297999999999998e-05, 'RS(255, 247)', 1.0323886639676114),
 (4.5, 0.0, 'RS(255, 247)', 1.0323886639676114)]

In [6]:
resultado.gera_vetor_snr()

[10.6, 9.8, 6.8, 5.3, 7.6, 6.6, 10.1, 5.7, 7.3, 8.3]

In [7]:
lista = resultado.busca_resultados(4.8)

In [ ]:
def get_maior_mos(tupla):
    return sorted(tupla, key=lambda element: element[0], reverse=True)

def get_menor_rate(tupla):
    return sorted(lista, key=lambda element: element[3])

def get_best(tupla):
    return sorted(sorted(lista, key=lambda element: element[3]),
                  key=lambda element: element[0], reverse=True)
    

In [ ]:
# Apartir daqui começa os testes
# retorna a lista ordenada com maior MOS e verifica se o maior mos é menor ou igual a 2.8
# se for retorna a lista ordenada de acordo com o maior rate

if sorted(lista, key=lambda element: element[0], reverse=True)[0][0] < 2.58:
    value = sorted(lista, key=lambda element: element[3])[0]
elif sorted(lista, key=lambda element: element[0], reverse=True)[0][0] > 4.03:
    value = sorted(lista, key=lambda element: element[3], reverse=True)[0]
else:
    value = sorted(sorted(lista, key=lambda element: element[3]),
                   key=lambda element: element[0], reverse=True)[0]
